Things to do:
////////////////make GND to 0 
////////////////removed dc/ac in voltage sources in netlist
////////////////read a netlist from a file remove unneccesary data --preprocessing
////////////////count number of componets and branches
////////////////make a dictionary for +node , -node,element type,value of elements
////////////////make a array of node numbers
////////////////make matrix G
////////////////make matrix voltage at nodes and current through those sources
////////////////make matrix for ind voltaage source and independent current source
////////////////solve the matricies
make a file input kind of system
!!!DO DC AC SIMUNTANEOUSLY


In [188]:
import numpy as np
#from sumpy import *
# processing the netlist
circ = """.circuit
R1 1 GND 1e3
R2 1 2 2e3
R3 2 3 4e3
V1 3 GND ac 2 0
V2 2 4 dc 5
R4 4 5 5e3
I1 5 GND dc 1
R6 4 6 8e3
I2 GND 6 dc 10
.end
.ac V1 50
""".splitlines()
# f = open(fn,'r')
# para = fd1.readlines().splitlines()

def delete_comments(circ):
    data = [' '.join(x.split()) for x in circ]            
    mode = circ[-1]
    data = [n for n in data if not n.startswith('.')] 
    if mode == '.end':
        return (0,'dc',data)
    else:
        k = mode.split()
        return (k[2],'ac', data)
    
(w0,mode,y) = delete_comments(circ)
w0 = int(w0)
data = []
for line in y:
    line = line.split(' ')
    if line[1] == 'GND':
        line[1] = '0'
    elif line[2] == 'GND':
        line[2] = '0'
    if line[3] == 'dc' or line[3] == 'ac':
        if line[3] == 'ac':
            ph=  line[5]
        del line[3]
    x = ' '.join(line)
    data.append(x)
data   

['R1 1 0 1e3',
 'R2 1 2 2e3',
 'R3 2 3 4e3',
 'V1 3 0 2 0',
 'V2 2 4 5',
 'R4 4 5 5e3',
 'I1 5 0 1',
 'R6 4 6 8e3',
 'I2 0 6 10']

In [189]:
#intializing variables
rlc_count = 0
v_count = 0
i_count = 0
#s = Symbol('t')

In [190]:
# number of components and branches
line_count = len(data) 
branch_count = 0

for i in range(line_count):
    comp = data[i][0]
    value_count = len(data[i].split())
    if (comp == 'R') or (comp == 'L') or (comp == 'C'):
        if value_count != 4:
            print("Error in netlist")
            print('Error in passive elements RLC')
        rlc_count += 1
        branch_count += 1
    elif comp == 'V':
        if value_count > 6:
            print("Error in netlist ")
            print('Error in Voltage source/node')
        v_count += 1
        branch_count += 1
    elif comp == 'I':
        if value_count != 4:
            print("Error in netlist ")
            print('Error in Current source/branch')
        i_count += 1
        branch_count += 1
    else:
        print('Unknow element found in netlist')
        


In [191]:
#data dictionary
data_dic = {'element':[],'+node':[],'-node':[],'value':[],'phase':[]}

def load_rlc(line_number):
    line = data[line_number].split()
    data_dic['element'] += [line[0]]
    data_dic['+node'] += [int(line[1])]
    data_dic['-node'] += [int(line[2])]
    data_dic['value'] += [float(line[3])]
    if line[0][0] == 'R':
        data_dic['phase'] += [0.0]
    elif line[0][0] == 'L':
        data_dic['phase'] += [np.pi/2]
    elif line[0][0] == 'C':
        data_dic['phase'] += [np.pi/2]
        
        
    
def load_sources(line_number):
    line = data[line_number].split()
    if mode == 'dc':
        data_dic['element'] += [line[0]]
        data_dic['+node'] += [int(line[1])]
        data_dic['-node'] += [int(line[2])]
        data_dic['value'] += [float(line[3])]
        data_dic['phase'] += [0.0]
    elif mode == 'ac':
            data_dic['element'] += [line[0]]
            data_dic['+node'] += [int(line[1])]
            data_dic['-node'] += [int(line[2])]
            data_dic['value'] += [float(line[3])]
            try:
                data_dic['phase'] += [float(line[4])]
            except IndexError:
                data_dic['phase'] += [0.0]
            
    else:
        return('Unknown mode')
        
    
        
    
# load_rlc(0)
# load_rlc(1)
# load_rlc(2)

In [192]:
#my_matrix = [([0]*cols) for i in range(rows)]

def count_nodes():
    n = [([0]*(line_count+1)) for i in range(1)]
    for i in range(line_count - 1):
        n[0][data_dic['+node'][i]] = data_dic['+node'][i]
        n[0][data_dic['-node'][i]] = data_dic['-node'][i]
        #largetst node
    if max(data_dic['-node']) > max(data_dic['+node']):
        largest = max(data_dic['-node'])
    else:
        largest =  max(data_dic['+node'])
        
    # check for unfilled elements, skip node 0
    for i in range(1,largest):
        if n[0][i] == 0:
            print('Error in node order')
    return largest

In [193]:
#load branch into dictionary
for i in range(line_count):
    comp = data[i][0]
    if (comp == 'R') or (comp == 'L') or (comp == 'C'):
        load_rlc(i)
    elif (comp == 'V') or (comp == 'I'):
        load_sources(i)
    else:
        print('Unknown Elelment Error')
        
# count number of nodes
num_nodes = count_nodes()

In [194]:
print(data_dic)
w0

{'element': ['R1', 'R2', 'R3', 'V1', 'V2', 'R4', 'I1', 'R6', 'I2'], '+node': [1, 1, 2, 3, 2, 4, 5, 4, 0], '-node': [0, 2, 3, 0, 4, 5, 0, 6, 6], 'value': [1000.0, 2000.0, 4000.0, 2.0, 5.0, 5000.0, 1.0, 8000.0, 10.0], 'phase': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}


50

In [195]:
#modify V and I to V*exp(j*ph) and I*exp(j*ph)


In [196]:
#make matricies
#V = np.zeros((num_nodes,1))
I = np.zeros((num_nodes,1),dtype='complex_')
#V = np.zeros((num_nodes,1),dtype = 'string_') 
G = np.zeros((num_nodes,num_nodes),dtype='complex_')
#s = Symbol('s')  # the Laplace variable

# count the number of element types that affect the size of the B, C, D, E and J arrays
k = v_count
if k != 0:
    B = np.zeros((num_nodes,k),dtype='complex_')
    C = np.zeros((k,num_nodes),dtype='complex_')
    D = np.zeros((k,k),dtype='complex_')
    E = np.zeros((k,1),dtype='complex_')
#    J = np.zeros((k,1),dtype='complex_')
C  

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [197]:
for i in range(branch_count):  # don't use branch count use # of rows in data frame
    n1 = data_dic['+node'][i]
    n2 = data_dic['-node'][i]
    # process all the passive elements, save conductance to temp value
    comp = data_dic['element'][i][0]   #get 1st letter of element name
    if comp == 'R':
        g = 1/data_dic['value'][i]
    if comp == 'L':
        if mode == 'ac':
            g = 1/(1j*w0*data_dic['value'][i])
        else:
            g = np.inf
    if comp == 'C':
        if mode == 'ac':
            g = 1j*w0*data_dic['value'][i]
        else:
            g = 0

    if (comp == 'R') or (comp == 'L') or (comp == 'C'):
        # If neither side of the element is connected to ground
        # then subtract it from appropriate location in matrix.
        if (n1 != 0) and (n2 != 0):
            G[n1-1,n2-1] += -g
            G[n2-1,n1-1] += -g

        # If node 1 is connected to ground, add element to diagonal of matrix
        if n1 != 0:
            G[n1-1,n1-1] += g

        # same for for node 2
        if n2 != 0:
            G[n2-1,n2-1] += g

print(G)

[[ 0.0015  +0.j -0.0005  +0.j  0.      +0.j  0.      +0.j  0.      +0.j
   0.      +0.j]
 [-0.0005  +0.j  0.00075 +0.j -0.00025 +0.j  0.      +0.j  0.      +0.j
   0.      +0.j]
 [ 0.      +0.j -0.00025 +0.j  0.00025 +0.j  0.      +0.j  0.      +0.j
   0.      +0.j]
 [ 0.      +0.j  0.      +0.j  0.      +0.j  0.000325+0.j -0.0002  +0.j
  -0.000125+0.j]
 [ 0.      +0.j  0.      +0.j  0.      +0.j -0.0002  +0.j  0.0002  +0.j
   0.      +0.j]
 [ 0.      +0.j  0.      +0.j  0.      +0.j -0.000125+0.j  0.      +0.j
   0.000125+0.j]]


In [198]:
# Current matrix containg current at each node
for i in range(branch_count):
    #n1 = df.loc[i,'p node']
    n1 = data_dic['+node'][i]
    n2 = data_dic['-node'][i]
    # process all the passive elements, save conductance to temp value
    comp = data_dic['element'][i][0]
    if comp == 'I':
        #g = data_dic['element'][i]
        g = data_dic['value'][i]*np.exp(1j*data_dic['phase'][i])
        #g = data_dic['value'][i]
        # sum the current into each node
        if n1 != 0:
            I[n1-1] = I[n1-1] - g
        if n2 != 0:
            I[n2-1] = I[n2-1] + g
            
print(I)

[[ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [-1.+0.j]
 [10.+0.j]]


In [199]:
rows, cols = num_nodes,1
V = [(''*1) for i in range(num_nodes)]
J = [(''*1) for i in range(v_count)]

In [200]:
# Voltage at each node
for i in range(num_nodes):
    V[i] = V[i] + 'v' +f'{i+1}'

V  # display the V matrix

['v1', 'v2', 'v3', 'v4', 'v5', 'v6']

In [201]:
# matrix J for current through voltage sources
sources = 0   # count source number
for i in range(branch_count):
    # process all the passive elements
    comp = data_dic['element'][i][0]  
    if comp == 'V':
        #J[sorces] = sympify('I_{:s}'.format(df.loc[i,'element']))
        J[sources] = 'I_{:s}'.format(data_dic['element'][i])
        sources += 1

J  # diplay the J matrix

['I_V1', 'I_V2']

In [202]:
# generate the B Matrix
# loop through all the branches and process independent voltage sources
sources = 0   # count source number
for i in range(branch_count):
    n1 = data_dic['+node'][i]
    n2 = data_dic['-node'][i]
    # process all the independent voltage sources
    #x = df.loc[i,'element'][0]   #get 1st letter of element name
    comp = data_dic['element'][i][0]
    if comp == 'V':
        if v_count > 1:
            if n1 != 0:
                B[n1-1][sources] = 1
            if n2 != 0:
                B[n2-1][sources] = -1
            sources += 1   #increment source count
        else:
            if n1 != 0:
                B[n1-1] = -1
            if n2 != 0:
                B[n2-1] = +1
        
B   # display the B matrix

array([[ 0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j],
       [ 1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j]])

In [203]:
# generate the C matrix
source = 0   # count source number
for i in range(branch_count):
    #n1 = df.loc[i,'p node']
    n1 = data_dic['+node'][i]
    n2 = data_dic['-node'][i]
    # process all the independent voltage sources
    #x = df.loc[i,'element'][0]   #get 1st letter of element name
    comp = data_dic['element'][i][0]
    if comp == 'V':
        if v_count > 1:
            if n1 != 0:
                C[source][n1-1] = 1
            if n2 != 0:
                C[source][n2-1] = -1
            source += 1   #increment source count
        else:
            if n1 != 0:
                C[0][n1-1] = -1
            if n2 != 0:
                C[0][n2-1] = +1
print(C)   # display the C matrix

[[ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j]]


In [204]:
print(D)

[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


In [206]:
# generate the E matrix
source = 0   # count source number
for i in range(branch_count):
    # process all the passive elements
    #x = df.loc[i,'element'][0]   #get 1st letter of element name
    comp = data_dic['element'][i][0]
    if comp == 'V':
        #E[source] = data_dic['element'][i]
        E[source] = data_dic['value'][i]*np.exp(1j*data_dic['phase'][i])
        source += 1

print(E)   # display the E matrix

[[2.+0.j]
 [5.+0.j]]


In [207]:
#matrix containg independent voltage source and current sources
#Z = I[:] + E[:]
#Z  # display the Z matrix
Z = np.concatenate((I,E))
print(Z.transpose())

[[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j 10.+0.j  2.+0.j  5.+0.j]]


In [208]:
#matrix containg node voltage and current through independent voltage sources
X = V[:] + J[:]
#X  # display the X matrix
# X = np.concatenate(V,J)
print(X)

['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'I_V1', 'I_V2']


In [209]:
#The A matrix is (m+n) by (m+n) and will be developed as the combination of 4 smaller matrices, G, B, C, and D.
n = num_nodes
m = v_count
A = np.zeros((m+n,m+n),dtype='complex_')
for i in range(n):
    for j in range(n):
        A[i,j] = G[i,j]

if v_count > 1:
    for i in range(n):
        for j in range(m):
            A[i,n+j] = B[i,j]
            A[n+j,i] = C[j,i]
else:
    for i in range(n):
        A[i,n] = B[i]
        A[n,i] = C[0][i]

A  # display the A matrix

array([[ 1.50e-03+0.j, -5.00e-04+0.j,  0.00e+00+0.j,  0.00e+00+0.j,
         0.00e+00+0.j,  0.00e+00+0.j,  0.00e+00+0.j,  0.00e+00+0.j],
       [-5.00e-04+0.j,  7.50e-04+0.j, -2.50e-04+0.j,  0.00e+00+0.j,
         0.00e+00+0.j,  0.00e+00+0.j,  0.00e+00+0.j,  1.00e+00+0.j],
       [ 0.00e+00+0.j, -2.50e-04+0.j,  2.50e-04+0.j,  0.00e+00+0.j,
         0.00e+00+0.j,  0.00e+00+0.j,  1.00e+00+0.j,  0.00e+00+0.j],
       [ 0.00e+00+0.j,  0.00e+00+0.j,  0.00e+00+0.j,  3.25e-04+0.j,
        -2.00e-04+0.j, -1.25e-04+0.j,  0.00e+00+0.j, -1.00e+00+0.j],
       [ 0.00e+00+0.j,  0.00e+00+0.j,  0.00e+00+0.j, -2.00e-04+0.j,
         2.00e-04+0.j,  0.00e+00+0.j,  0.00e+00+0.j,  0.00e+00+0.j],
       [ 0.00e+00+0.j,  0.00e+00+0.j,  0.00e+00+0.j, -1.25e-04+0.j,
         0.00e+00+0.j,  1.25e-04+0.j,  0.00e+00+0.j,  0.00e+00+0.j],
       [ 0.00e+00+0.j,  0.00e+00+0.j,  1.00e+00+0.j,  0.00e+00+0.j,
         0.00e+00+0.j,  0.00e+00+0.j,  0.00e+00+0.j,  0.00e+00+0.j],
       [ 0.00e+00+0.j,  1.00e+00+0.j,  0.

In [210]:
print(np.linalg.solve(A,Z))

[[ 5.14314286e+03+0.j]
 [ 1.54294286e+04+0.j]
 [ 2.00000000e+00+0.j]
 [ 1.54244286e+04+0.j]
 [ 1.04244286e+04+0.j]
 [ 9.54244286e+04+0.j]
 [ 3.85685714e+00+0.j]
 [-9.00000000e+00-0.j]]


refrences: https://lpsa.swarthmore.edu/Systems/Electrical/mna/MNAAll.html